In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
dataset = pd.read_csv("../../data/intermediante_files/raw_dataset_reindexed.csv")

# Stockout Identification

In [3]:
def calculate_intermitency(value_list):

    intermitency = []

    for idx, el in enumerate(value_list):
        if idx == 0:
            if el == 0:
                intermitency.append(1)
            else:
                intermitency.append(0)
        else:
            if el != 0:
                intermitency.append(0)
            else:
                intermitency.append(intermitency[idx-1]+1)

    return intermitency

def identify_stockout(df, sales_column, window, threshold, verbose=0):
    from statistics import NormalDist
    from numpy import nan
    
    # step 1 calculate intermitency and rolling stats
    df["intermitency"] = calculate_intermitency(df[sales_column].tolist())
    df["intermitency_mean"] = df["intermitency"].rolling(window=window, min_periods=0, center=False).mean()
    df["intermitency_std"]  = df["intermitency"].rolling(window=window, min_periods=0, center=False).std()
    
    #step 2 propagate distribution parameters in case of zero observed sales
    df[["intermitency_mean", "intermitency_std"]] = df[["intermitency_mean", "intermitency_std"]].fillna(0)
    df["intermitency_mean"] = df[[sales_column,"intermitency_mean"]].apply(lambda row: row.intermitency_mean if row[sales_column] != 0 else nan, axis=1)
    df["intermitency_std"]  = df[[sales_column,"intermitency_std"]].apply(lambda row: row.intermitency_std if row[sales_column] != 0 else nan, axis=1)
    df[["intermitency_mean", "intermitency_std"]] = df[["intermitency_mean", "intermitency_std"]].fillna(method = "ffill")
    
    # step 3 calculate probability and identify stockouts
    df["prob"] = df.apply(lambda row: NormalDist(mu=row.intermitency_mean, sigma=max(row.intermitency_std, 0.001)).cdf(row.intermitency), axis=1)
    df["is_stockout"] = df.apply(lambda x: 1 if (x[sales_column] == 0.0  and x.prob > 1-threshold) else 0, axis=1)
    
    if not verbose:
        df = df.drop(["intermitency", "intermitency_mean", "intermitency_std", "prob"], axis=1)
    
    return df

# Sales correction

In [4]:
def correct_sales(df, sales_col, stockout_column, window, verbose=0):
    
    smoothed_col = "{}_smoothed".format(sales_col)
    corrected_col = "{}_corrected".format(sales_col)
    
    df[smoothed_col] = df[sales_col].rolling(window = window, min_periods = 0, center = True).mean()
    df[corrected_col] = df.apply(lambda row: row[smoothed_col] if row[stockout_column] == 1 else row[sales_col], axis=1)
    
    if not verbose:
        df = df.drop([sales_column, smoothed_col], axis=1)
    
    return df

# Correct stockout and sales

In [5]:
res = []
for store_nbr in dataset.store_nbr.unique():
    print(store_nbr, end="\r")
    sub_dataset = dataset.loc[(dataset.store_nbr==store_nbr)]
    sub_dataset = sub_dataset.groupby(["store_nbr", "item_nbr"], as_index=False).apply(lambda df: identify_stockout(df, "unit_sales", 30, 0.01, verbose=1))
    sub_dataset = sub_dataset.groupby(["store_nbr", "item_nbr"], as_index=False).apply(lambda df: correct_sales(df, "unit_sales", "is_stockout", df.intermitency.max(), verbose=1))
    res.append(sub_dataset)

In [6]:
for idx, ds in enumerate(res):
    print("exporting {}".format(idx))
    ds.to_csv("../../data/intermediante_files/chunk_{}.csv".format(idx), index=False)

exporting 0
exporting 1
exporting 2
exporting 3
exporting 4
exporting 5
exporting 6
exporting 7
exporting 8
exporting 9
exporting 10
exporting 11
exporting 12
exporting 13
exporting 14
exporting 15
exporting 16
exporting 17
exporting 18
exporting 19
exporting 20
exporting 21
exporting 22
exporting 23
exporting 24
exporting 25
exporting 26
exporting 27
exporting 28
exporting 29
exporting 30
exporting 31
exporting 32
exporting 33
exporting 34
exporting 35
exporting 36
exporting 37
exporting 38
exporting 39
exporting 40
exporting 41
exporting 42
exporting 43
exporting 44
exporting 45
exporting 46
exporting 47
exporting 48
exporting 49
exporting 50
exporting 51
exporting 52
exporting 53


In [7]:
res = []
for idx in range(54):
    print("importing {}".format(idx))
    ds = pd.read_csv("../../data/intermediante_files/chunk_{}.csv".format(idx))
    res.append(ds)

importing 0
importing 1
importing 2
importing 3
importing 4
importing 5
importing 6
importing 7
importing 8
importing 9
importing 10
importing 11
importing 12
importing 13
importing 14
importing 15
importing 16
importing 17
importing 18
importing 19
importing 20
importing 21
importing 22
importing 23
importing 24
importing 25
importing 26
importing 27
importing 28
importing 29
importing 30
importing 31
importing 32
importing 33
importing 34
importing 35
importing 36
importing 37
importing 38
importing 39
importing 40
importing 41
importing 42
importing 43
importing 44
importing 45
importing 46
importing 47
importing 48
importing 49
importing 50
importing 51
importing 52
importing 53


In [8]:
new_ds = pd.concat(res)

In [9]:
new_ds.to_csv("../../data/intermediante_files/dataset_corrected.csv", index=False)